In [6]:
#!/usr/bin/env python
import pika
import psycopg2
import json

'''
TODO

Table_fields

  run_id -> incremental por run
 ,generation 
 ,variables
 ,fitness

CREATE TABLE IF NOT EXISTS
'''

credentials = pika.PlainCredentials('guest', 'guest')
connection = pika.ConnectionParameters('rabbitmq',
                                       5672,
                                       '/',
                                       credentials)

connection = pika.BlockingConnection(connection)
channel = connection.channel()

channel.queue_declare(queue='testando')

conn = psycopg2.connect(host="postgres",
                        database="genetico", 
                        user="postgres", 
                        password="Postgres2019!")

cur = conn.cursor()

def create_table(name):
    create_table_query = '''CREATE TABLE IF NOT EXISTS {}
      (RUN            TEXT    NOT NULL,
       GENERATION      INT    NOT NULL,
       VARIABLES       TEXT    NOT NULL,
       FITNESS       NUMERIC(7,6)    NOT NULL); '''.format(name)
    
    cur.execute(create_table_query)
    conn.commit()
    print("Table created successfully in PostgreSQL ")
    
    
def callback(ch, method, properties, text):
    sql = """INSERT INTO initial(run, generation, variables, fitness)
             VALUES(%s, %s, %s, %s);"""
    text= json.loads(str(text, 'utf-8'))
    cur.execute(sql, (text['run'], text['generation'], text['variables'], text['fitness'],))
    conn.commit()
    
channel.basic_consume(queue='testando',
                      auto_ack=True,
                      on_message_callback=callback)

create_table('Initial')

Table created successfully in PostgreSQL 


In [7]:
print(' [*] Waiting for messages. To exit press CTRL+C')
channel.start_consuming()

 [*] Waiting for messages. To exit press CTRL+C


StreamLostError: Stream connection lost: ConnectionResetError(104, 'Connection reset by peer')